In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
ds = pd.read_csv('./mushrooms.csv')
print ds.shape
ds.head()

(8124, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
le = LabelEncoder()
df = ds.apply(le.fit_transform)
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [14]:
print type(df)
print type(df.loc)
split = int(0.9*df.values.shape[0])
print split
train = df.loc[:split]
test = df.loc[split:]
print type(train)
print train.shape
print test.shape
test.head()
test = test.reset_index().drop('index', axis=1)
test.head()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.indexing._LocIndexer'>
7311
<class 'pandas.core.frame.DataFrame'>
(7312, 23)
(813, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,5,0,8,0,5,1,1,0,2,...,2,7,7,0,2,2,4,7,3,1
1,1,3,3,2,0,8,1,0,1,0,...,2,7,6,0,2,1,0,7,4,2
2,1,3,3,4,0,7,1,0,1,0,...,1,7,7,0,2,1,0,7,4,4
3,0,5,2,3,0,5,1,1,0,7,...,2,7,7,0,2,2,4,7,2,1
4,0,5,2,8,0,5,1,1,0,2,...,1,7,7,0,2,2,4,7,2,1


In [21]:
print df.shape

print len(df['type'])
print type(df['type'])

x = np.unique(df['type'], return_counts=True)
print x
count = 0
for ix in x[1]:
    print "Type ", x[0][count], " ", ix/float(x[1].sum())
    count+=1

(8124, 23)
8124
<class 'pandas.core.series.Series'>
(array([0, 1]), array([4208, 3916]))
Type  0   0.517971442639094
Type  1   0.48202855736090594


In [30]:
def prior_prob(data, label_column, label_value):
    n_rows = data.loc[data[label_column]==label_value]
    #print n_rows.shape
    return n_rows.shape[0]/float(data.shape[0])

print prior_prob(train,'type', 1)

def cond_prob(data, feature_column, feature_value, label_column, label_value):
    
    constraint = data.loc[(data[label_column]==label_value) &
                         (data[feature_column]==feature_value)]
    #print constraint.shape
    n_rows = data.loc[data[label_column]==label_value]
    return constraint.shape[0]/float(n_rows.shape[0])

print cond_prob(train, 'cap_shape', 5, 'type', 0)

0.473605032823
0.476747207067


In [31]:
## Naive Bayes Implementation Loop
preds = []
CLASSES = np.unique(test['type'])
print CLASSES
print test.shape
count = 0
for ix in range(test.shape[0]):
    if count%100==0:
        print count
    count+=1
    val = test.loc[ix][1:]
    features = dict(val)
    posterior = []
    for cx in CLASSES:
        likelihood = 1.0
        for fx in features:
            cp = cond_prob(train, fx, features[fx], 'type', cx)
            likelihood *= cp
        
        prior = prior_prob(train, 'type', cx)
        posterior_prob = prior*likelihood
        posterior.append(posterior_prob)
    
    prediction = np.array(posterior).argmax()
    preds.append(prediction)

[0 1]
(813, 23)
0
100
200
300
400
500
600
700
800


In [28]:
value = test.loc[0][1:]
print dict(value)

{'stalk_root': 0, 'stalk_color_above_ring': 7, 'stalk_surface_above_ring': 2, 'gill_attachment': 1, 'gill_spacing': 1, 'habitat': 1, 'stalk_shape': 0, 'spore_print_color': 7, 'gill_size': 0, 'gill_color': 2, 'ring_number': 2, 'veil_type': 0, 'odor': 5, 'stalk_color_below_ring': 7, 'cap_surface': 0, 'ring_type': 4, 'population': 3, 'veil_color': 2, 'stalk_surface_below_ring': 2, 'bruises': 0, 'cap_color': 8, 'cap_shape': 5}


In [40]:
ground_truth_labels = test['type']
print ground_truth_labels.shape

preds = np.array(preds)
print preds.shape

compare_out = np.array(preds==ground_truth_labels)
print compare_out.shape
print compare_out[:10].sum()

accuracy = compare_out.sum()/float(compare_out.shape[0])
print accuracy

(813,)
(813,)
(813,)
10
0.97539975399754


In [33]:
print accuracy

0.97539975399754
